# Test WaffleIron Model

In [2]:
from models.waffleiron.segmenter import Segmenter

In [3]:
model = Segmenter(
    input_channels=5,
    feat_channels=768,
    depth=48,
    grid_shape=[[256, 256], [256, 32], [256, 32]],
    nb_class=1024,
    #drop_path_prob=config["waffleiron"]["drop_path"],
    layer_norm=True,
)

In [ ]:
# Load pretrained model
ckpt = torch.load(args.pretrained_ckpt, map_location="cpu")
if ckpt.get("model_points") is not None:
    ckpt = ckpt["model_points"]
else:
    ckpt = ckpt["model_point"]
new_ckpt = {}
for k in ckpt.keys():
    if k.startswith("module"):
        new_ckpt[k[len("module.") :]] = ckpt[k]
    else:
        new_ckpt[k] = ckpt[k]
model.classif = torch.nn.Conv1d(
    config["waffleiron"]["nb_channels"], config["waffleiron"]["pretrain_dim"], 1
)
model.load_state_dict(new_ckpt)